In [4]:
import pandas as ps
import numpy as nm
import math as mt
import copy

class Node :
    def __init__( self, attribute_name, data, attribute_values, select_from ) :
        self.attribute_name = attribute_name
        self.data = data 
        self.attribute_values = attribute_values
        self.childrens = []
        self.select_from = select_from
    def add_child( self, child_obj ):
        self.childrens.append( child_obj )   

def value_entropy( lable, att, val, data_u ):
    att_col = ps.Series( data_u[ att ] ).tolist()
    lable_col = ps.Series( data_u[ lable ] ).tolist() 
    count1 = 0
    count = 0
    i = 0
    proT = 0
    proF = 0
    while( i < len( lable_col ) ):#Counting size and number of ones for that(val) value of this attribute
        if( att_col[ i ] == val ) :
            count = count + 1
            if( lable_col[ i ] == 1 ) :
                count1 = count1 + 1 #for numbmer of ones
        i = i + 1         

    proT = count1 / count
    proF = 1 - proT
    if( count1 == count or count1 == 0):
        entropy = 0
    else :    
        entropy = ( proT * mt.log( proT )/mt.log( 2 ) + proF * mt.log( proF )/mt.log( 2 ) ) * ( -1 )
    
    return entropy        

def calc_entropy( curr_data, lable ) :
    lable_col = ps.Series( curr_data[ lable ] )
    count1 = 0
    count = 0
    pro1 = 0 
    pro0 = 0
    
    for val in lable_col :
        if( val == 1 ) :
            count1 = count1 + 1
        count = count + 1
        
    pro1 = count1 / count
    pro0 = 1 - pro1
    if( count1 == count or count1 == 0 ) :
        entropy = 0
    else :    
        entropy = ( pro1 * ( mt.log( pro1 ) / mt.log( 2 ) ) + pro0 * ( mt.log( pro0 ) / mt.log( 2 ) ) ) * ( -1 )
    
    return entropy

def select_attribute( curr_data, select_from, parent_entropy ):
    p_row, p_col = curr_data.shape
    ff = 0
    gain_ratio = -10
    for attribute in select_from :
        att_size = p_row #For first attribute size of attribute will be size of dataset
        val_size = []#to store size of class after split using any particular value of that attribute
        val_ent = []#to store entropy of class after split using any particular value of that attribute
        vals_attribute = curr_data[ attribute ].unique()#getting all unique values of a particular col
        for val in vals_attribute :
            c_row, c_col = curr_data[ curr_data[ attribute ] == val ].shape
            val_size.append( c_row )
            val_ent.append( value_entropy( lable, attribute, val, curr_data ) )

        info = 0
        i = 0
        while( i < len( val_size ) ) :#information calculation
            temp = val_size[ i ] / att_size
            temp = temp * val_ent[ i ]
            info = info + temp
            i = i + 1
        gain = parent_entropy - info# gain calculated

        intrinsic_info = 0
        i = 0
        temp = 0
        while( i < len( val_size ) ) :#intinsic info calculation
            temp = val_size[ i ] / att_size 
            temp = temp * ( mt.log( temp ) / mt.log( 2 ) ) 
            intrinsic_info = intrinsic_info + temp
            i = i + 1 
        if( intrinsic_info == 0 ):
            intrinsic_info = 1
        else:
            intrinsic_info = intrinsic_info * ( -1 )
    
        gr = ( gain / intrinsic_info )

        if( gr > gain_ratio or ff == 0 ) :
            gain_ratio = gr 
            attribute_selected = attribute
            if( ff == 0 ) :
                ff = 1 

    return attribute_selected


    

In [5]:
data = ps.read_csv("Training.csv") #here data is DataFrame object
row, col = data.shape
print("Rows ", row, "Columns ", col )
col_names = list( data.columns.values )
lable = "left"

to_be_selected = []
i = 0

# while( i < col ) :#Running while loop for each column to filter out categoric attribute
#     if( col_names[ i ] != lable and len( data[ col_names[ i ] ].unique() ) <= 10 ):   
#         to_be_selected.append( col_names[ i ] )    
#     i = i + 1   
to_be_selected = [ "Work_accident", "promotion_last_5years", "Department", "salary" ] 

Entropy_Def = calc_entropy( data, lable )
#entropy for first attribute
#Entropy of first attribute will be same irrespective of attribute we are selecting for it, 
#as data available for it is the whole dataset

ff = 0 #to make first gain ratio enter correctly
gain_ratio = 0
attribute_selected = select_attribute( data, to_be_selected, Entropy_Def )


to_be_selected.remove( attribute_selected )
root = Node( attribute_selected, data, data[ attribute_selected ].unique(), to_be_selected ) #Creating root node
print( "Root : ", root.attribute_name, "\nValues ", root.attribute_values)

nodes = []

nodes.append( root )

while( len( nodes ) > 0 ) :
    current = nodes.pop( 0 )
    
    attribute = current.attribute_name
#     print( "\n\nParent ", attribute )
    curr_data = current.data
    select_from = copy.deepcopy( current.select_from )
    values = current.attribute_values
    curr_entropy = calc_entropy( curr_data, lable )
    if( curr_entropy != 0 and len( select_from ) > 0 ) :
        for val in values :
            node_data = curr_data[ curr_data[ attribute ] == val ]
            attribute_selected = select_attribute( node_data, select_from, curr_entropy )#current data, list of attributes from selection to be made,
                                                                                       #categoric attributes with their all possible values
            node_select_from = copy.deepcopy( select_from )
            node_select_from.remove( attribute_selected )
            child = Node( attribute_selected, node_data, node_data[ attribute_selected ].unique(), node_select_from )
            current.add_child( child )
            if( calc_entropy( node_data, lable ) != 0 ):
                nodes.append( child )
#             print( "Child ", attribute_selected, "for value ", val )
        if( len( node_select_from ) != 0 ) :
            current.data.drop( columns = current.data.columns )        


Rows  8993 Columns  10
Root :  Work_accident 
Values  [0 1]


In [6]:
def no_val( c_node, row_i ) :
    count_rows = 0
    count_ones = 0
    child = 1
    val_no = 0
    for val in c_node.childrens :
        next_node = val
        while( child == 1 ) : #Traversing whole tree
#             print( "Node's attribute ", next_node.attribute_name, ", with values ", next_node.attribute_values )
            val_row_i = row_i[ next_node.attribute_name ] #Value in row red for attribute, the node is of( attribute on which node is defined )
#             print("Value in input row ", val_row_i )

            if( len( next_node.childrens ) > 0 ):
                if val_row_i in next_node.attribute_values.tolist() :
                    ind = next_node.attribute_values.tolist().index( val_row_i )
                    next_node = next_node.childrens[ ind ]
                else :
                    rt = no_val( next_node )
                    val_no = 1
                    child = 0
            else :
                child = 0
         
        n_row, n_col = next_node.data.shape
    
        if( val_no == 1 ) :
            if( rt == 1 ) :
                count_ones = count + 1
            count.rows = count.rows + 1
        elif( n_row == 1 ) : #Calculating Prediction
            res_row = next_node.data.iloc[ 0 ]
            if( res_row[ lable ] == 1 ) :
                count_ones = count_ones + 1
            count_rows + 1
        else:
            j = 0
            count_1 = 0
            count_t = 0
            while( j < n_row ) :
                resm_row = next_node.data.iloc[ j ]
                if( resm_row[ lable ] == 1 ) :
                    count_1 = count_1 + 1
                count_t = count_t + 1
                j = j + 1

            if( count_1 / count_t  >= 0.5 ) :
                count_ones = count_ones + 1
            count_rows = count_rows + 1
            
    if( count_ones / count_rows >= 0.5 ) :
        rt = 1
    else :
        rt = 0
        
    return rt


validation_data = ps.read_csv("Valid.csv") #here data is DataFrame object
v_row, v_col = validation_data.shape
print("Rows :", v_row, ", Columns : ", v_col)

i = 0
A_yes_P_no = 0
A_yes_P_yes = 0
A_no_P_no = 0
A_no_P_yes = 0

while( i < v_row ) :
    row_i = validation_data.iloc[ i ]
    child = 1
    next_node = root 
#     print( "\n\n New test row ", i + 1 )
    val_no = 0
    while( child == 1 ) : #Traversing whole tree
#         print( "Node's attribute ", next_node.attribute_name, ", with values ", next_node.attribute_values )
        val_row_i = row_i[ next_node.attribute_name ] #Value in row red for attribute, the node is of( attribute on which node is defined )
#         print("Value in input row ", val_row_i )
                
        if( len( next_node.childrens ) > 0 ):#Checking if the node has childrens or not
            if val_row_i in next_node.attribute_values.tolist() :#Checking if the value of row is in node or not( it can be null as well in that case it won't be in node )
                #If row value is not in the node, then we go to all childrens and check for next attribute( node ) and recursively check for it
                ind = next_node.attribute_values.tolist().index( val_row_i )
                next_node = next_node.childrens[ ind ]
            else :
                rt = no_val( next_node, row_i )
                val_no = 1
                child = 0
        else :
            child = 0
    
    n_row, n_col = next_node.data.shape
    
    if( val_no == 1 ) :
        if( rt == 1 ) :
            if( row_i[ lable ] == 1 ) :
                A_yes_P_yes = A_yes_P_yes + 1
            else :
                A_no_P_yes = A_no_P_yes + 1
        else:
            if( row_i[ lable ] == 0 ) :
                A_no_P_no = A_no_P_no + 1
            else :
                A_yes_P_no = A_yes_P_no + 1
    
    elif( n_row == 1 ) : #Calculating Prediction
        res_row = next_node.data.iloc[ 0 ]
        if( res_row[ lable ] == 1 ) :
            if( row_i[ lable ] == 1 ) :
                A_yes_P_yes = A_yes_P_yes + 1
            else :
                A_no_P_yes = A_no_P_yes + 1
        else:
            if( row_i[ lable ] == 0 ) :
                A_no_P_no = A_no_P_no + 1
            else :
                A_yes_P_no = A_yes_P_no + 1
                    
    else:
        j = 0
        count_1 = 0
        count_t = 0
        while( j < n_row ) :
            resm_row = next_node.data.iloc[ j ]
            if( resm_row[ lable ] == 1 ) :
                count_1 = count_1 + 1
            count_t = count_t + 1
            j = j + 1
       
        if( count_1 / count_t  >= 0.5 ) :
            if( row_i[ lable ] == 1 ) :
                A_yes_P_yes = A_yes_P_yes + 1
            else :
                A_no_P_yes = A_no_P_yes + 1
        else :
            if( row_i[ lable ] == 0 ) :
                A_no_P_no = A_no_P_no + 1
            else :
                A_yes_P_no = A_yes_P_no + 1
        
        
    i =  i + 1
    
print("Number of rows red : ", i )

print(" Actual yes Predicted yes ", A_yes_P_yes )    
print(" Actual No Predicted yes ", A_no_P_yes )
print(" Actual yes Predicted no ", A_yes_P_no )
print(" Actual_no_Predicted_no", A_no_P_no )
# print( data )

Rows : 2245 , Columns :  10
Number of rows red :  2245
 Actual yes Predicted yes  0
 Actual No Predicted yes  0
 Actual yes Predicted no  543
 Actual_no_Predicted_no 1702
